In [ ]:
# !pip install onnx==1.7.0
# # !pip install tensorflow==1.15.0
# !pip install torch==1.4.0
# !pip install onnx_tf==1.5.0

In [ ]:
import tensorflow as tf
tf.__version__


In [ ]:
# !pip uninstall tensorflow

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
import pickle
from collections import Counter


class ModelDataLoader(Dataset):
    
    def __init__(self, utt2LabelDictPath, utt2VectorDictPath, utt2AttDictPath):
        assert(utt2LabelDictPath != None)
        assert(utt2VectorDictPath != None)
        assert(utt2AttDictPath != None)
        try :
            with open(utt2LabelDictPath, 'rb') as handle:
                self.utt2LabelDict = pickle.load(handle)
            with open(utt2VectorDictPath, 'rb') as handle:
                self.utt2VectorDict = pickle.load(handle)
            with open(utt2AttDictPath, 'rb') as handle:
                self.utt2AttDict = pickle.load(handle)
        except :
            raise Exception('Invalid path')
            
        self.len = len(self.utt2LabelDict.keys())
        self.idx2Utt = {}
        self.lab2LabIdMap = {}
        self.Labels = []
        
        for idx, utt in enumerate(self.utt2LabelDict.keys()):
            self.idx2Utt[idx] = utt
            self.Labels.append(self.utt2LabelDict[utt])
        for labId, lab in enumerate(set(self.Labels)):
            self.lab2LabIdMap[lab] = labId
            
    def __len__(self):
        return self.len
    
    def getNumClasses(self):
        return len(Counter(self.Labels))
    
    def __getitem__(self, idx):
        utt = self.idx2Utt[idx]
        vec = self.utt2VectorDict[utt]
        lab = self.utt2LabelDict[utt]
        labId = self.lab2LabIdMap[lab]
        if utt in self.utt2AttDict:
            att = self.utt2AttDict[utt]
        else:
            att = [0]*52
        return vec, utt, labId

In [ ]:
import torch
import tensorflow as tf
import onnx
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters BASELINE
input_size = 512
hidden_size = 220
num_classes = 27
num_epochs = 25
batch_size = 40
learning_rate = 0.001

# from onnx_tf.backend import prepare


In [ ]:
#MNIST dataset 
bw = "bow_guse_"
lg = "en_"
# lg = "fr_"
# lg = "de_"
# ty = "sf_"
ty = "it_"
# ty = "hr_"
c = bw + lg + ty



train_dataset = DataLoader(c + 'train_uttlab.pickle', c + 'train_uttvec.pickle', c + 'train_uttatt.pickle') 
test_dataset = DataLoader(c +'test_uttlab.pickle', c + 'test_uttvec.pickle', c + 'train_uttatt.pickle') 
# print(len(Counter(list(train_dataset.lab2LabIdMap.values()))))
# print(len(Counter(list(test_dataset.lab2LabIdMap.values()))))

# comb = train_dataset.Labels + test_dataset.Labels
# comb = list(Counter(comb))
# newmap = {}
# for i in range(len(comb)):
#     newmap[comb[i]] = i
    
# train_dataset.lab2LabIdMap = newmap    
# test_dataset.lab2LabIdMap = newmap    

# train_dataset.lab2LabIdMap.update(test_dataset.lab2LabIdMap)
# test_dataset.lab2LabIdMap = train_dataset.lab2LabIdMap



# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=True)

eval_train = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=4000, 
                                           shuffle=True)

eval_test = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=4000, 
                                          shuffle=True)
print(len(train_dataset))
print(len(test_dataset))
print(len(Counter(list(train_dataset.lab2LabIdMap.values()))))
print(len(Counter(list(test_dataset.lab2LabIdMap.values()))))

In [ ]:
# print(list(train_dataset.utt2AttDict.values())[0:5])
# print(list(train_dataset.utt2AttDict.keys())[0:5])
# print(list(train_dataset.utt2LabelDict.values())[0:5])

def ang(x,y):
    nx = np.linalg.norm(x)
    ny = np.linalg.norm(y)
    cos = np.dot(x, y)/(nx * ny)
    if cos > 1:
        cos = 1
    elif cos < -1:
        cos = -1
    return 1 - np.arccos(cos)/np.pi
ind1 = 800
ind2 = 12
print(list(train_dataset.utt2VectorDict.keys())[ind1])
print(list(train_dataset.utt2VectorDict.keys())[ind2])
vec1 = list(test_dataset.utt2VectorDict.values())[ind1]
vec2 = list(test_dataset.utt2VectorDict.values())[ind2]
print(vec1.shape)
ang(vec1, vec2)

In [ ]:
# one = 46
# two = 44
# a = list(test_dataset.utt2VectorDict.values())[one]
# b = list(test_dataset.utt2VectorDict.values())[two]
# print(list(test_dataset.utt2VectorDict.keys())[one])
# print(list(test_dataset.utt2VectorDict.keys())[two])
# print(list(test_dataset.utt2LabelDict.values())[one])
# print(list(test_dataset.utt2LabelDict.values())[two])
# def ang(x,y):
#     nx = np.linalg.norm(x)
#     ny = np.linalg.norm(y)
#     cos = np.dot(x, y)/(nx * ny)
#     if cos > 1:
#         cos = 1
#     elif cos < -1:
#         cos = -1
#     return 1 - np.arccos(cos)/np.pi

# print(ang(a, b))
# print(list(test_dataset.utt2LabelDict.keys())[1:50])

In [ ]:

def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph
    

In [ ]:
###BASELINE
###NUM CLASSES

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


if not os.path.exists('./ZS/'):
    os.mkdir('./ZS/')

if not os.path.exists('./ZSModels/'):
    os.mkdir('./ZSModels/')    

if not os.path.exists('./ZSGraphs/'):
    os.mkdir('./ZSGraphs/')  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (vectors, utts, labId) in enumerate(train_loader):  
        # Move tensors to the configured device
        vectors = vectors.to(device)

        
        outputs = model(vectors)
        loss = criterion(outputs, labId)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 5 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
#             if epoch > 3.0:
#                 title = './RNN/' + str(epoch+1) + "." + str(i+1) + '.onnx'
#                 dummy_input = torch.rand(1, 1, 512)
#                 dummy_output = model.emb(dummy_input)
#                 torch.onnx.export(model, dummy_input, title, input_names=['test_input'], output_names=['test_output'])
                                
#                 graph_title = './RNNGraphs/' + str(epoch+1) + "." + str(i+1) + '.pb'
#                 model_onnx = onnx.load(title)
#                 tf_rep = prepare(model_onnx)
#                 tf_rep.export_graph(graph_title)
                
#                 model_title = './RNNModels/' + str(epoch+1) + "." + str(i+1)
#                 tf_graph = load_pb(graph_title)
                
#                 with tf.Session(graph=tf_graph) as sess:
#                     sess.run(tf.global_variables_initializer())
#                     ops = tf.get_default_graph().get_operations() 
#                     sess.run(tf.get_default_graph().get_operation_by_name('init'))

#                     ou_tensor = tf_graph.get_tensor_by_name('test_output:0')
#                     in_tensor = tf_graph.get_tensor_by_name('test_input:0')

#                     tf.saved_model.simple_save(sess, model_title, inputs={'input': in_tensor}, outputs={'output': ou_tensor})

                

In [ ]:
print(list(train_dataset.utt2VectorDict.keys())[0:8])
print(list(train_dataset.utt2LabelDict.values())[0:8])



In [ ]:
###BASELINE
###NUM CLASSES


from sklearn.metrics import f1_score
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
vector, atts, labId = [], [], []
vec, att, lab = [], [], []
with torch.no_grad():
    for i, (vec, att, lab) in enumerate(eval_test):
        outputs = model(vec.to(device))
        _, predicted = torch.max(outputs.data, 1)
    a = f1_score(lab, predicted, average='weighted')
    print("Trained: " + str(a))
    
    
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
vector, atts, labId = [], [], []
vec, att, lab = [], [], []
with torch.no_grad():
    for i, (vectors, atts, labId) in enumerate(eval_train):
        neighbor = KNeighborsClassifier(n_neighbors=1)
        neighbor.fit(vectors.numpy(), labId.numpy())
    for i, (vec, att, lab) in enumerate(eval_test):
        predicted = neighbor.predict(vec.numpy())
    b = f1_score(lab, predicted, average='weighted')
    print("KNN: " + str(b))


In [ ]:
path = "BoW_Models/" + c + str(a)
torch.save(model.state_dict(), path)

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(torch.load(path))
model.eval()